Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [1]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [4]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [5]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
#network.add(layer(ActFunc.hb, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [6]:
# pos = np.random.rand(31)

# weights = pos[0:27]
# weights_arr = []
# j = 0
# for i in range(len(network.layers)):
#     weight_mat = np.reshape(weights[j:j + (network.layers[i].numOfNodes * 8)], (network.layers[i].numOfNodes, -1))
#     weights_arr.append(weight_mat)
#     j = j + (network.layers[i].numOfNodes * 8)
#     input = network.layers[i].numOfNodes


# bias = pos[27:31] # Total num of nodes in the NN
# bias_arr = []
# j = 0
# for i in range(len(network.layers)):
#     bias_arr.append(bias[j:j + network.layers[i].numOfNodes])
#     j = j + network.layers[i].numOfNodes

# weights_arr

In [7]:
# yhat = X_demo.apply(network.forwardCalculation, args = (weights_arr, bias_arr), axis = 1)
# network.sseCalculation(yhat, y_demo)

In [12]:
# PSO test!
import pso

swarmsize = 10
alpha = 0.5
beta = 1
gamma = 1
delta = 1.5
epsilon = 0.4

particle_swarm_opti = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon)
opti_particle = particle_swarm_opti.optimise()



Current best = 2302.0116696596447
Current best = 1664.4051467139054
Current best = 1371.0432223544879
Current best = 1286.5409767602987
Current best = 1251.1225062234907
Current best = 1210.2680738151628
1210.2680738151628


In [9]:
# print(pso.particles)
# for i in range(10):
#     print(pso.particles[i].informant)


In [10]:
# pso.particles[i].informant[0].pos

In [ ]:
# Testing the optimisation
# Ooooo bad bad result :D
print(opti_particle.pos)
particle_swarm_opti.assessFitness(opti_particle.pos, y_train)

1201.9153250344375